In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from matplotlib import patches

def manacher_algorithm(s, visualize=False):
    # Step 1: Transform the input string
    T = '#'.join(f"^{s}$")
    n = len(T)
    P = [0] * n
    center = right = 0
    steps = []
    #Step 2: Loop through the transformed string
    for i in range(1, n - 1):
        mirror = 2 * center - i
        #  Step 3: If within the right boundary, set initial palindrome radius
        if i < right:
            P[i] = min(right - i, P[mirror])
            # Step 4: Expand the palindrome around the current character
        while T[i + 1 + P[i]] == T[i - 1 - P[i]]:
            P[i] += 1
        if i + P[i] > right:
            center, right = i, i + P[i]
        if visualize:
            steps.append((i, T[i], P[i], T[i - P[i]:i + P[i] + 1].replace('#', '')))

    max_len, center_index = max((n, i) for i, n in enumerate(P))
    start = (center_index - max_len) // 2
    return s[start: start + max_len], steps

# Widgets
input_text = widgets.Text(value='', placeholder='Enter a string here', description='Input:', disabled=False)
output_text = widgets.Output()
step_button = widgets.Button(description='Next Step', button_style='warning', icon='step-forward')
all_steps_button = widgets.Button(description='Show All Steps', button_style='success', icon='play')
find_button = widgets.Button(description='Find Palindrome', button_style='info', icon='search')
progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', bar_style='info')
result_area = widgets.HTML(value="", layout=widgets.Layout(margin='10px 0'))

# Global variables to track steps
current_step = 0
steps = []

def on_button_clicked(b):
    global current_step, steps
    with output_text:
        clear_output()
        if not input_text.value:
            result_area.value = "<b style='color:red;'>Please enter a string.</b>"
            return

        if b.description == 'Find Palindrome':
            result, steps = manacher_algorithm(input_text.value, visualize=True)
            current_step = 0
            progress_bar.max = len(steps)
            progress_bar.value = 0
            result_area.value = f"<b style='color:green;'>Longest Palindromic Substring: {result}</b>"
            # Colorful, transparent visualization
            fig, ax = plt.subplots(figsize=(19, 4))
            ax.text(0.5, 0.5, result, size=40, ha='center', va='center',
                    bbox=dict(boxstyle="round,pad=0.3", edgecolor='none', facecolor='green', alpha=0.5))
            ax.set_aspect('equal')
            ax.axis('off')
            plt.show()

        elif b.description == 'Next Step':
            if current_step < len(steps):
                i, char, radius, sub = steps[current_step]
                result_area.value = f"<b>Step {current_step+1}:</b> Center at index {i} ('{char}'), Radius: {radius}, Substring: '{sub}'"
                current_step += 1
                progress_bar.value = current_step
                visualize_step(input_text.value, steps[current_step - 1])
            else:
                result_area.value = "<b style='color:blue;'>No more steps to show.</b>"

        elif b.description == 'Show All Steps':
            all_steps_output = ""
            for idx, step in enumerate(steps):
                i, char, radius, sub = step
                all_steps_output += f"<b>Step {idx+1}:</b> Center at index {i} ('{char}'), Radius: {radius}, Substring: '{sub}'<br>"
            result_area.value = all_steps_output

def visualize_step(s, step):
    i, char, radius, sub = step
    T = '#'.join(f"^{s}$")
    fig, ax = plt.subplots(figsize=(15, 4))
    ax.text(0.5, 0.5, f"Current Step: Center at {i} ('{char}')", size=20, ha='center', va='center')
    for j in range(len(T)):
        ax.text(j, 0.7, T[j], size=15, ha='center', va='center', color='blue' if j == i else 'black')
    ax.add_patch(patches.Rectangle((i-radius, 0.4), 2*radius, 0.2, edgecolor='red', facecolor='none'))
    ax.set_xlim(0, len(T)-1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    plt.show()

# Linking buttons to the event handler
find_button.on_click(on_button_clicked)
step_button.on_click(on_button_clicked)
all_steps_button.on_click(on_button_clicked)

# Displaying widgets in a structured layout
ui = widgets.VBox([
    input_text,
    widgets.HBox([find_button, step_button, all_steps_button]),
    progress_bar,
    output_text,
    result_area
])

display(ui)
